In [16]:
import tensorflow.compat.v1 as tf

In [2]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
deep_learning = tf.constant('Deep Learning')
session = tf.Session()

In [8]:
session.run(deep_learning)

b'Deep Learning'

In [5]:
x = tf.placeholder(tf.float32, name='x', shape=[None, 784])
W = tf.Variable(tf.random_uniform([784,10], -1, 1), name='W')
multiply = tf.matmul(x, W)

<font size=5> hello <br>
it's **me**. </font><br>
I'll be fine.

In [17]:
# 복잡한 모델을 만들 때는 인스턴스화 하고 싶은 변수들의 집합을 한 곳에서부터 재사용하고 공유해야할 때가 많다.
def my_network(input):
    W_1 = tf.Variable(tf.random_uniform([784, 100], -1, 1), name='W_1')
    b_1 = tf.Variable(tf.zeros([100]), name='biases_1')
    output_1 = tf.matmul(input, W_1) + b_1 # z = (x * w) + b : Logit을 구하는 과정
    
    W_2 = tf.Variable(tf.random_uniform([100, 50], -1, 1), name='W_2')
    b_2 = tf.Variable(tf.zeros([50]), name='biases_2')
    output_2 = tf.matmul(output_1, W_2) + b_2
    
    W_3 = tf.Variable(tf.random_uniform([50, 10], -1, 1), name='W_3')
    b_3 = tf.Variable(tf.zeros([10]), name='biases_3')
    output_3 = tf.matmul(output_2, W_3) + b_3
    
    # 이름 출력
    print("Printing names of weight parameters")
    print(f"{W_1.name} {W_2.name} {W_3.name}")
    print("Printing names of bias parameters")
    print(f"{b_1.name} {b_2.name} {b_3.name}")
    
    return output_3

In [18]:
# 그런데 두 개의 다른 입력으로 위 신경망을 사용하려고 시도하면...? 예상치못한 일이 발생한다.
i_1 = tf.placeholder(tf.float32, [1000, 784], name='i_1')

In [19]:
my_network(i_1)

Printing names of weight parameters
W_1:0 W_2:0 W_3:0
Printing names of bias parameters
biases_1:0 biases_2:0 biases_3:0


<tf.Tensor 'add_2:0' shape=(1000, 10) dtype=float32>

In [20]:
# 다른 입력 placeholder 인스턴스로 위 신경망을 다시 한 번 사용해보자.
i_2 = tf.placeholder(tf.float32, [1000, 784], name='i_2')

In [21]:
my_network(i_2)

Printing names of weight parameters
W_1_1:0 W_2_1:0 W_3_1:0
Printing names of bias parameters
biases_1_1:0 biases_2_1:0 biases_3_1:0


<tf.Tensor 'add_5:0' shape=(1000, 10) dtype=float32>

<font size=5> 오잉...? 이름이 이상해. 처음 사용했던 변수들과 이름이 달라? <br>
즉, 서로 다른 입력인스턴스의 동일한 신경망 파라미터 변수를 사용하려고 할시, <br>
사실상 동일한 변수를 사용하지 않는 것이다. <br>
이름을 보아도 알겠지만, 서로 다르다! </font>

<font size=5> 이게 무슨 말이냐면, **입력할 때마다** 사본들을 계속 만들어내면서 <br>
전혀 다른 변수집합들을 계속 만들어 사용한다는 것이다.<br>
대부분의 경우에는 이처럼 파라미터 변수들이 매번 사본을 만들어 사용하는 것이 아니라,<br>
동일한 변수들을 계속 재사용하길 원하는데(왜냐면 training을 해야하니까...), <br>
**이 경우에는 tf.Variable을 사용해서는 안 된다**.<br>
그 대신 **텐서플로의 변수 범위 지정**을 이용한 더욱 진보한 명명 체계를 사용해야한다.</font>

In [22]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_uniform_initializer(minval=-1, maxval=1)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    return tf.matmul(input, W) + b

def my_network(input):
    with tf.variable_scope("layer_1"):
        output_1 = layer(input, [784, 100], [100])

    with tf.variable_scope("layer_2"):
        output_2 = layer(output_1, [100, 50], [50])
    
    with tf.variable_scope("layer_3"):
        output_3 = layer(output_2, [50, 10], [10])
        
    return output_3

In [23]:
i_1 = tf.placeholder(tf.float32, [1000, 784], name='i_1')
my_network(i_1)

<tf.Tensor 'layer_3/add:0' shape=(1000, 10) dtype=float32>

In [25]:
i_2 = tf.placeholder(tf.float32, [1000, 784], name='i_2')
my_network(i_2) # 에러 발생! 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zoshs2/anaconda3/envs/tf_tutorial/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-443af6ca2414>", line 2, in <module>
    my_network(i_2)
  File "<ipython-input-22-aaada42b925f>", line 10, in my_network
    output_1 = layer(input, [784, 100], [100])
  File "<ipython-input-22-aaada42b925f>", line 4, in layer
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/variable_scope.py", line 1572, in get_variable
    aggregation=aggregation)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/variable_scope.py", line 1315, in get_variable
    aggregation=aggregation)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/variable_scope.py", line 568, in get_variable
    aggre

ValueError: Variable layer_1/W already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 1756, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3322, in _create_op_internal
    op_def=op_def)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/op_def_library.py", line 742, in _apply_op_helper
    attrs=attr_protos, op_def=op_def)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/gen_state_ops.py", line 1527, in variable_v2
    shared_name=shared_name, name=name)
  File "/home/zoshs2/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/state_ops.py", line 79, in variable_op_v2
    shared_name=shared_name)


In [26]:
# 이처럼 tf.Variable 과는 달리 tf.get_variable 명령어는 주어진 이름의 변수가 인스턴스화 되지 않았는지를 확인해본다.
# 기본적으로 이처럼 공유는 허용되지 않지만(안정성을 위해서), 
# 변수 범위 내에서 공유될 수 있게 하려면 다음과 같이 명시적으로 지정할 수 있다.
with tf.variable_scope("shared_variables") as scope:
    i_1 = tf.placeholder(tf.float32, [1000, 784], name='i_1')
    my_network(i_1)
    scope.reuse_variables()
    i_2 = tf.placeholder(tf.float32, [1000, 784], name='i_2')
    my_network(i_2)

## 3.9 CPU & GPU로 모델 관리

<font size=3> 계산 그래프가 어떤 장치들을 사용하는지 검사하기 위해<br>
**log_device_placement=True**로 설정해 텐서플로 세션을 다음과 같이 초기화 할 수 있다.

In [27]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0



<font size=3> 여기 gate2 에서는 CPU, GPU커널이 모두 있는 상황이여서 <br>
**/cpu:0** : 컴퓨터의 CPU<br>
**/gpu:0** : 컴퓨터의 첫 번째 GPU<br>
가 모두 조회된다. <br>
그리고 GPU가 게다가 **사용가능한 상태**이므로, 텐서플로는 알아서 GPU사용을 자동으로 선택한다.</font>

In [111]:
# 특정 (GPU or CPU)장치를 사용하고 싶다면, tf.device 로 적절한 장치를 선택할 수도 있다.
# 하지만 선택한 장치가 사용불가능한 상태라면, 오류를 발생시킨다.
# (Tip!) 선택한 장치가 '존재하지 않을 때'는 텐서플로에서 사용가능한 다른 장치를 알아서 찾아낼 수도 있다.
# 이를 수행하기 위해서는 세션에 allow_soft_placement플래그를 True로 설정하면 된다.
with tf.device('/device:GPU:2'): # GPU 1번 사용하기
    a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2, 2], name='a')
    b = tf.constant([1.0, 2.0], shape=[2, 1], name='b')
    c = tf.matmul(a, b)
    

In [114]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0



In [115]:
sess.run(c)

array([[ 5.],
       [11.]], dtype=float32)

<font size=3> 또한 텐서플로는 **"타워 방식(Tower-like Fashion)"**의 모델을 작성해서 <br>
**여러 GPU를 사용하는 모델**을 만들 수 있다. <br>

In [82]:
# Tower-like Fashion model
c = []

for d in ['/gpu:0', '/gpu:1']:
    with tf.device(d):
        a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2, 2], name='a')
        b = tf.constant([1.0, 2.0], shape=[2, 1], name='b')
        c.append(tf.matmul(a, b))
        
with tf.device('/cpu:0'):
    sum = tf.add_n(c)
    
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0



In [83]:
sess.run(sum)

array([[10.],
       [22.]], dtype=float32)

# 우선 여기서 마무리하고 다음 노트(ipynb)에서 GPU를 어떻게하면 똑똑하게 사용하는지 알아보도록 하자